In [1]:
# IMPORT CODE 
import os 
import cv2
import numpy as np
import pickle

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import *
from sklearn.model_selection import RandomizedSearchCV
from IPython import display
from PIL import Image


def cv2_imshow(a):
  """A replacement for cv2.imshow() for use in Jupyter notebooks.
  Args:
    a : np.ndarray. shape (N, M) or (N, M, 1) is an NxM grayscale image. shape
      (N, M, 3) is an NxM BGR color image. shape (N, M, 4) is an NxM BGRA color
      image.
  """
  a = a.clip(0, 255).astype('uint8')
  # cv2 stores colors as BGR; convert to RGB
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display.display(Image.fromarray(a))

def load_data(location, labels, metric):
    fv_list = os.listdir(location)
    fv_nb = len(fv_list)

    with open(location + fv_list[0], "rb") as handle: 
        fv0 = pickle.load(handle)

    with open(labels, "rb") as handle: 
        labels = pickle.load(handle)

    print("Number of vectors: " + str(fv_nb) + " of length " + str(len(fv0)))

    data_x = np.zeros(shape=(fv_nb, len(fv0)))
    data_y = np.zeros(shape=(fv_nb,))

    for i, fv_filename in enumerate(fv_list): 
        with open(location + fv_filename, "rb") as handle: 
            fv = pickle.load(handle)
        
        fid = fv_filename.split(".")[0]

        data_y[i] = 0
        if metric in labels[fid]: 
            data_y[i] = labels[fid][metric] 

        data_x[i] = fv

    return data_x, data_y, fv_list

In [7]:
DATA_FOLDER = "D:\\NIST datasets\\feature_vector_dataset\\"

LABELS = DATA_FOLDER + "qualities_v2.pkl"
metric = "nfiq2"

train_x, train_y, train_names = load_data(DATA_FOLDER + "train/", LABELS, metric)
test_x, test_y, test_names = load_data(DATA_FOLDER + "test/", LABELS, metric)

Number of vectors: 12000 of length 192
Number of vectors: 1200 of length 192


In [8]:
import pickle
regr = MLPRegressor(verbose=True, alpha=0.0001, random_state=2, max_iter=500, solver="adam")
regr.fit(train_x, train_y)

predictions = np.clip(regr.predict(test_x), 0, 100)
train_predictions = np.clip(regr.predict(train_x), 0, 100)

print("train MSE", mean_squared_error(train_y, train_predictions))
print("test MSE", mean_squared_error(test_y, predictions))

print("train MAE", mean_absolute_error(train_y, train_predictions))
print("test MAE", mean_absolute_error(test_y, predictions))

print("train R2", r2_score(train_y, train_predictions))
print("test R2", r2_score(test_y, predictions))

with open("nn_model_nfq.pkl", "wb") as handle: 
    pickle.dump(regr, handle)

Iteration 1, loss = 26437.83219675
Iteration 2, loss = 225.13156772
Iteration 3, loss = 101.89072367
Iteration 4, loss = 64.95964125
Iteration 5, loss = 52.67105414
Iteration 6, loss = 45.81471989
Iteration 7, loss = 41.04711390
Iteration 8, loss = 38.52202595
Iteration 9, loss = 36.95691145
Iteration 10, loss = 35.76409176
Iteration 11, loss = 35.34854960
Iteration 12, loss = 34.86419530
Iteration 13, loss = 34.29140994
Iteration 14, loss = 34.66523103
Iteration 15, loss = 32.73559394
Iteration 16, loss = 32.54549698
Iteration 17, loss = 32.45623406
Iteration 18, loss = 32.13887360
Iteration 19, loss = 32.70223988
Iteration 20, loss = 31.25108522
Iteration 21, loss = 31.02934685
Iteration 22, loss = 30.94360868
Iteration 23, loss = 30.79683477
Iteration 24, loss = 30.76941389
Iteration 25, loss = 30.43099183
Iteration 26, loss = 30.09478630
Iteration 27, loss = 29.44487339
Iteration 28, loss = 29.44848969
Iteration 29, loss = 29.95462505
Iteration 30, loss = 28.89720248
Iteration 31, 